In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import correlation
from tqdm import tqdm
import data_preprocessing;

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maksat.kengeskanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/maksat.kengeskanov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
folder = "files/"
clusters_w_sw = pd.read_csv(folder + "clusterized_tweets_with_stopwords.csv")
clusters_no_sw = pd.read_csv(folder + "clusterized_tweets_without_stopwords.csv")
sentiments_w_sw = pd.read_csv(folder + "sentiment_twitters_numerical.csv").iloc[:, 1:]
stocks = pd.read_csv(folder + "clean_stocks.csv")
# sentiments_no_sw = pd.read_csv(folder + )
clusters_w_sw.date = pd.to_datetime(clusters_w_sw.date)
clusters_no_sw.date = pd.to_datetime(clusters_no_sw.date)
sentiments_w_sw.date = pd.to_datetime(sentiments_w_sw.date)
# sentiments_no_sw.date = pd.to_datetime(sentiments_no_sw.date)
stocks.date = pd.to_datetime(stocks.date)

In [6]:
def combine_dataframes(clusters, sentiments):
    merged = pd.merge(clusters, sentiments, on = ['date','tweet', 'language', 
    'hashtags', 'cashtags',
    'link', 'retweet', 'nlikes', 
    'nreplies', 'nretweets', 'reply_to'], how = 'inner')
    merged = data_preprocessing.cut_dates(merged)
    merged['interactions'] = merged['nlikes'] + merged['nreplies'] + merged['nretweets']
    merged = merged.drop(columns=['nlikes', 'nreplies', 'nretweets'])
    popular = merged.groupby('date').max('interactions').merge(merged[['date', 'tweet']], on = 'date', how = 'right')
    popular.date = popular.date.dt.normalize()
    joined = pd.merge(popular, stocks, on = 'date', how = 'inner')
    return joined

In [7]:
combined_w_sw = combine_dataframes(clusters_w_sw, sentiments_w_sw)

,date,retweet,cluster_use_default,cluster_mpnet_default,cluster_minilm_default,cluster_distilbert_default,cluster_use_dbcv,cluster_mpnet_dbcv,cluster_minilm_dbcv,cluster_distilbert_dbcv,...,sentiment,bertwitter_sentiment,hf_sentiment,interactions,tweet,open,high,low,close,adj
0,2011-12-01,False,-1,-1,-1,7,-1,1,0,-1,...,1,1,1,236,Went to Iceland on Sat to ride bumper cars on ...,6.514,6.7980,6.3960,6.520,6.520
1,2011-12-01,False,2,5,-1,10,1,1,0,2,...,0,0,-1,118,I made the volume on the Model S go to . Now I...,6.514,6.7980,6.3960,6.520,6.520
2,2011-12-03,False,-1,9,-1,-1,-1,1,0,1,...,1,1,1,61,"Great Voltaire quote, arguably better than Twa...",6.636,6.8690,6.5830,6.772,6.772
3,2011-12-03,False,6,-1,-1,-1,2,1,0,-1,...,0,0,-1,90,That was a total non sequitur btw,6.636,6.8690,6.5830,6.772,6.772
4,2011-12-04,False,6,-1,-1,21,2,1,0,-1,...,1,1,1,91,Am reading a great biography of Ben Franklin b...,6.671,6.9345,6.6345,6.828,6.828


In [8]:
methods_w_sw = ['cluster_use_default', 'cluster_mpnet_default', 'cluster_minilm_default', 'cluster_distilbert_default', 
'cluster_use_dbcv', 'cluster_mpnet_dbcv', 'cluster_minilm_dbcv', 'cluster_distilbert_dbcv']
cramer_w_sw = correlation.find_correlation(combined_w_sw, methods_w_sw).sort_values('cramer', ascending=False).reset_index(drop = True)
cramer_w_sw

,method,cluster,type,cramer,percent_change,delay
0,cluster_distilbert_default,7,sentiment,0.41622,1,1
1,cluster_distilbert_default,9,bertwitter_sentiment,0.36535,0,1
2,cluster_minilm_default,0,sentiment,0.35717,0,3
3,cluster_minilm_default,24,sentiment,0.35390,0,1
4,cluster_minilm_default,14,sentiment,0.35203,1.5,3
...,...,...,...,...,...,...
3262,cluster_use_default,5,bertwitter_sentiment,0.00000,2,1
3263,cluster_use_default,5,bertwitter_sentiment,0.00000,0,2
3264,cluster_minilm_default,29,hf_sentiment,0.00000,0,3
3265,cluster_minilm_default,29,hf_sentiment,0.00000,0,2


In [9]:
from clustering import top_ten_words_v2 as tt 

cramer_w_sw[['size', 'common_words', 'counts']] = ''
for i in tqdm(range(0, cramer_w_sw.shape[0])):
    check = combined_w_sw.loc[combined_w_sw[cramer_w_sw.method[i]] == cramer_w_sw.cluster[i]]
    top_ten = tt(check.tweet, cramer_w_sw.cluster[i])
    cramer_w_sw.at[i, 'size'] = top_ten['Size'][0]
    cramer_w_sw.at[i, 'common_words'] = top_ten['Common words'][0]
    cramer_w_sw.at[i, 'counts'] = top_ten['Counts'][0]
cramer_w_sw

100%|██████████| 3267/3267 [3:16:27<00:00,  3.61s/it]


,method,cluster,type,cramer,percent_change,delay,size,common_words,counts
0,cluster_distilbert_default,7,sentiment,0.41622,1,1,32,"[texas, la, south, beautiful, place, weather, ...","[5, 5, 4, 4, 3, 3, 3, 3, 2, 2]"
1,cluster_distilbert_default,9,bertwitter_sentiment,0.36535,0,1,34,"[neuralink, amp, neural, brain, yes, neuro, sk...","[18, 8, 6, 6, 5, 5, 5, 4, 4, 4]"
2,cluster_minilm_default,0,sentiment,0.35717,0,3,40,"[id, aa, carmack, soumiksf, would, must, hard,...","[39, 39, 39, 4, 3, 2, 2, 2, 2, 2]"
3,cluster_minilm_default,24,sentiment,0.35390,0,1,41,"[solar, power, energy, solarcity, per, day, el...","[36, 19, 18, 7, 6, 5, 5, 5, 4, 4]"
4,cluster_minilm_default,14,sentiment,0.35203,1.5,3,46,"[ppathole, gary, garythebot, yes, snail, theon...","[27, 9, 8, 6, 5, 2, 2, 2, 2, 2]"
...,...,...,...,...,...,...,...,...,...
3262,cluster_use_default,5,bertwitter_sentiment,0.00000,2,1,52,"[haha, lol, ahem, evafoxu, sigh, trouble, nich...","[30, 4, 4, 3, 2, 2, 2, 2, 1, 1]"
3263,cluster_use_default,5,bertwitter_sentiment,0.00000,0,2,52,"[haha, lol, ahem, evafoxu, sigh, trouble, nich...","[30, 4, 4, 3, 2, 2, 2, 2, 1, 1]"
3264,cluster_minilm_default,29,hf_sentiment,0.00000,0,3,61,"[welcome, happy, thanks, littlemissflint, good...","[11, 10, 8, 7, 6, 6, 4, 4, 4, 4]"
3265,cluster_minilm_default,29,hf_sentiment,0.00000,0,2,61,"[welcome, happy, thanks, littlemissflint, good...","[11, 10, 8, 7, 6, 6, 4, 4, 4, 4]"
